###### ДЗ 1 Классификация

*Креативная часть:* найти парамеры и комбинации их, которые будут увеличивать accuracy

*Техническая часть:* сделать под это всё удобные обёртки. Например, это функция, которая просто принимает список фичей, а на выходе показывает результат фита

In [11]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model, model_selection
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt

In [3]:
def logReg(df, X, Y):
    
    #выделяем данные для регрессии
    alldata = df[X + Y]
    #смешиваем
    alldata = alldata.sample(frac=1)
    
    #определеяем параметры входа и выхода
    allinput = alldata[X]
    alloutput = alldata[Y]
    
    #делим выборку на тестовую и обучающую
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(allinput, alloutput, test_size=0.2)
    
    #строим модель
    model = linear_model.LogisticRegression()
    #обучаем модель
    model.fit(X_train, Y_train.values.ravel())
    
    #рассчитываем долю правильных ответов
    accuracy = model.score(X_test, Y_test)
    
    #print(accuracy)
    
    return accuracy

#модель предсказывает значения для тестовых данных
    #Y_test_predicted = model.predict(X_test)

In [2]:
def mean_accuracy(df, X, Y):
    a = np.array([])
    for i in range(100):
        a = np.append(a, logReg(df, X, Y))
    print(a.mean())

In [31]:
def show_scatter(x,y, xLabel, yLabel):
    plt.xlabel(xLabel)
    plt.ylabel(yLabel)    
    plt.scatter(x, y)
    plt.plot(x, np.poly1d(np.polyfit(x, y, 1))(x),"r--")
    plt.show()

### Готовим данные для использования функции

Я решила, что будет удобно сначала преобразовать Y в бинарный тип и добавить его сразу к df, -- чтобы извлекать данные внутри функции.

In [6]:
df_sochi = pd.read_csv("./athletes_sochi.txt").dropna()
df_sochi

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland
...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga Pavlin,98.0,0,0,0,0,Ice Hockey,Slovenia
2853,31,1982-12-05,Female,1.70,Zina Kocher,60.0,0,0,0,0,Biathlon,Canada
2854,28,1985-06-14,Female,1.68,Zoe Gillings,65.0,0,0,0,0,Snowboard,Great Britain
2856,22,1991-03-01,Male,1.76,Zongyang Jia,68.0,0,0,1,1,Freestyle Skiing,China


In [8]:
dfs = df_sochi.copy()
gn = df_sochi["gender"].apply(lambda x: int(x == "Male"))
dfs["gender_num"] = gn
dfs

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country,gender_num
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States,1
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy,1
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan,1
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand,1
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga Pavlin,98.0,0,0,0,0,Ice Hockey,Slovenia,1
2853,31,1982-12-05,Female,1.70,Zina Kocher,60.0,0,0,0,0,Biathlon,Canada,0
2854,28,1985-06-14,Female,1.68,Zoe Gillings,65.0,0,0,0,0,Snowboard,Great Britain,0
2856,22,1991-03-01,Male,1.76,Zongyang Jia,68.0,0,0,1,1,Freestyle Skiing,China,1


### Добавляем фичи

Индекс массы тела

In [9]:
dfs["BMI"] = dfs["weight"] / (dfs["height"]**2)

Вид спорта

In [7]:
ord_enc = OrdinalEncoder()

In [8]:
dfs['sport_num'] = ord_enc.fit_transform(dfs[["sport"]])

Страна

In [9]:
dfs['country_num'] = ord_enc.fit_transform(dfs[["country"]])

In [10]:
dfs

,age,birthdate,gender,height,name,weight,gold_medals,silver_medals,bronze_medals,total_medals,sport,country,gender_num,BMI,sport_num,country_num
0,17,1996-04-12,Male,1.72,Aaron Blunck,68.0,0,0,0,0,Freestyle Skiing,United States,1,22.985398,5.0,79.0
1,27,1986-05-14,Male,1.85,Aaron March,85.0,0,0,0,0,Snowboard,Italy,1,24.835646,12.0,36.0
2,21,1992-06-30,Male,1.78,Abzal Azhgaliyev,68.0,0,0,0,0,Short Track,Kazakhstan,1,21.461937,9.0,39.0
4,21,1992-07-30,Male,1.86,Adam Barwood,82.0,0,0,0,0,Alpine Skiing,New Zealand,1,23.702162,0.0,54.0
5,21,1992-12-18,Male,1.75,Adam Cieslar,57.0,0,0,0,0,Nordic Combined,Poland,1,18.612245,8.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2851,28,1985-04-30,Male,1.93,Ziga Pavlin,98.0,0,0,0,0,Ice Hockey,Slovenia,1,26.309431,6.0,67.0
2853,31,1982-12-05,Female,1.70,Zina Kocher,60.0,0,0,0,0,Biathlon,Canada,0,20.761246,1.0,12.0
2854,28,1985-06-14,Female,1.68,Zoe Gillings,65.0,0,0,0,0,Snowboard,Great Britain,0,23.030045,12.0,27.0
2856,22,1991-03-01,Male,1.76,Zongyang Jia,68.0,0,0,1,1,Freestyle Skiing,China,1,21.952479,5.0,15.0


### Пытаемся улучшить accuracy........

Отталкиваться будем от модели, построенной на паре

В рассчетах используется среднее значение accuracy, так как итеративно оно поддается достаточно сильным изменениям, настолько сильным, что судить об изменении предсказательных способностей модели не представляется возможным.

Но метод подсчета среднего значения также не очень эффективен, верить оценке можно только до двух знаков после запятой.

In [30]:
mean_accuracy(dfs, ['weight', 'height'], ["gender_num"])

0.8568145161290321


Добавим третью фичу:

In [29]:
mean_accuracy(dfs, ['weight', 'height', 'BMI'], ["gender_num"])

0.8661491935483872


In [32]:
mean_accuracy(dfs, ['weight', 'height', 'sport_num'], ["gender_num"])

0.8594758064516131


In [33]:
mean_accuracy(dfs, ['weight', 'height', 'country_num'], ["gender_num"])

0.8527016129032259


In [34]:
mean_accuracy(dfs, ['weight', 'height', 'age'], ["gender_num"])

0.8525403225806452


#### Вывод:

BMI позволяет более точно определять пол спортсменов, а остальные параметры нет.